### 5.
O processamento de informação geográfica é uma ferramenta omnipresente no nosso dia a dia. Pretende-se
estudar treinos associados a vários desportos. Foi criado um dataset associados a vários utilizadores da aplicação
EndoMondo. Serão disponibilizado vários ficheiros json onde será possível explorar a dinâmica dos vários treinos.
Pretende-se que sejam explorados usando a biblioteca ipyleaflet. Pretende-se que seja desenhadas as etapas,
calculados o nº de km a subir e a descer, determinada a velocidade média (em todo o percurso e separadamente
nos troços a subir e a descer), a distância em plano, etc. Devem comparar a evolução do ritmo cardíaco em
vários percursos e relaciona-lo com as zonas de treino.

In [ ]:
from ipyleaflet import Map, basemaps, GeoJSON
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import json
import os

def create_map(longitudes, latitudes):
    coordinates = list(zip(latitudes, longitudes))
    start_line = coordinates[0]
    m = Map(center=start_line, zoom=10, basemap=basemaps.OpenStreetMap.HOT)
    geojson = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [[lon, lat] for lat, lon in coordinates],
        },
    }
    line = GeoJSON(data=geojson, style={"color": 'red', "weight": 2})
    m.add_layer(line)
    display(m)
    
def plot_heart_rate(heart_rates, timestamps):
    time_minutes = [(t - timestamps[0]) / 60 for t in timestamps]
    plt.figure(figsize=(10, 5))
    plt.plot(time_minutes, heart_rates, label='Frequência Cardíaca', color='blue')
    plt.title('Frequência Cardíaca ao Longo do Tempo')
    plt.xlabel('Tempo (minutos)')
    plt.ylabel('Frequência Cardíaca (bpm)')
    plt.grid()
    plt.legend()
    plt.show()
    
def analyze_route(workout, latitudes, longitudes, altitudes, timestamps, heart_rates, gender, sport_type):
    ascent, descent = 0, 0
    dist_ascent, dist_descent, dist_flat = 0, 0, 0
    time_ascent, time_descent, time_flat = 0, 0, 0
    total_dist = 0
    total_time = 0

    for i in range(1, len(latitudes)):
        alt_diff = altitudes[i] - altitudes[i - 1]
        time_diff = timestamps[i] - timestamps[i - 1]
        d = total_distance((latitudes[i - 1], longitudes[i - 1]), (latitudes[i], longitudes[i]))
        total_dist += d
        total_time += time_diff
        if alt_diff > 1:
            ascent += alt_diff
            dist_ascent += d
            time_ascent += time_diff
        elif alt_diff < -1:
            descent += abs(alt_diff)
            dist_descent += d
            time_descent += time_diff
        else:
            dist_flat += d
            time_flat += time_diff

    hours = int(total_time // 3600) 
    minutes = int((total_time % 3600) // 60)
    seconds = int(total_time % 60)
    formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}"
    speed_ascent = (dist_ascent/1000) / (time_ascent / 3600) if time_ascent > 0 else 0
    speed_descent = (dist_descent/1000) / (time_descent / 3600) if time_descent > 0 else 0
    speed_flat = (dist_flat/1000) / (time_flat / 3600) if time_flat > 0 else 0
    lthr = calculate_lthr(workout, timestamps, heart_rates)
    for i in range(len(heart_rates)):
        hr = heart_rates[i]
        if workout.get('sport') == 'run':
            zone = classify_zone(hr, lthr, 'run')
        elif workout.get('sport') == 'bike':
            zone = classify_zone(hr, lthr, 'bike')
        else:
            zone = 'Desconhecida'
        
    print(f"Treino: {workout.get('sport', 'Desconhecido')}")
    print(f"Genero: {workout.get('gender', 'Desconhecido')}")
    print(f"Zona de ritmo: {zone}")
    print(f"Tempo total: {formatted_time}")
    print(f"Subida total: {ascent:.2f} m")
    print(f"Descida total: {descent:.2f} m")
    print(f"Distância subida: {dist_ascent:.2f} m")
    print(f"Distância descida: {dist_descent:.2f} m")
    print(f"Distância plano: {dist_flat:.2f} m")
    print(f"Velocidade subida: {speed_ascent:.2f} km/h")
    print(f"Velocidade descida: {speed_descent:.2f} km/h")
    print(f"Velocidade plano: {speed_flat:.2f} km/h")
    hours = total_time / 3600.0
    dist_km = total_dist / 1000.0
    avg_speed_total = dist_km / hours if hours > 0 else 0
    print(f"Velocidade média total: {avg_speed_total:.2f} km/h")
    print(f"Distancia total: {total_dist:.2f} m")
    print("")
    
def total_distance(p1, p2):
    return geodesic(p1, p2).meters

def calculate_lthr(workout, timestamps, heart_rates):
    if len(timestamps) < 30 or len(heart_rates) < 30:
        return None
    
    lap_time = timestamps[0] + 10 * 60

    data_20_minutes = [
        hr for i, hr in enumerate(heart_rates) if timestamps[i] >= lap_time
    ]

    if len(data_20_minutes) == 0:
        return None
    
    lthr = sum(data_20_minutes) / len(data_20_minutes)
    return round(lthr)

def classify_zone(heart_rate, lthr, sport_type):
    if sport_type == 'run':
        if heart_rate < 0.85 * lthr:
            return 'Zona 1'
        elif heart_rate <= 0.89 * lthr:
            return 'Zona 2'
        elif heart_rate <= 0.94 * lthr:
            return 'Zona 3'
        elif heart_rate <= 0.99 * lthr:
            return 'Zona 4'
        elif heart_rate <= 1.02 * lthr:
            return 'Zona 5a'
        elif heart_rate <= 1.06 * lthr:
            return 'Zona 5b'
        elif heart_rate > 1.06 * lthr:
            return 'Zona 5c'
    elif sport_type == 'bike':
        if heart_rate < 0.81 * lthr:
            return 'Zona 1'
        elif heart_rate <= 0.89 * lthr:
            return 'Zona 2'
        elif heart_rate <= 0.93 * lthr:
            return 'Zona 3'
        elif heart_rate <= 0.99 * lthr:
            return 'Zona 4'
        elif heart_rate <= 1.02 * lthr:
            return 'Zona 5a'
        elif heart_rate <= 1.06 * lthr:
            return 'Zona 5b'
        elif heart_rate > 1.06 * lthr:
            return 'Zona 5c'
    
workouts_data = {}
for file_number in range(50):
    json_filename = f'DatasetEndoMondo/endomondoHR_proper_subset{file_number}.json'
    try:
        with open(json_filename, 'r', encoding='utf-8') as f:
            file_data = json.load(f)
            workouts_data.update(file_data)
    except FileNotFoundError:
        print(f"O ficheiro {json_filename} não foi encontrado.")
    except json.JSONDecodeError:
        print(f"Erro ao decodificar JSON no ficheiro {json_filename}.")
    
workout_ids = list(workouts_data.keys())

if workout_ids:
    workout_id = '392337038'  # Use the ID directly
    if workout_id in workouts_data:
        workout = workouts_data[workout_id]

    print(f"\n--- Dados do treino (ID: {workout_id}) ---")
    print(f"Desporto: {workout.get('sport', 'Não especificado')}")
    print(f"ID d0 utilizador: {workout.get('userId', 'Não especificado')}")
    print(f"Género do utilizador: {workout.get('gender', 'Não especificado')}")

    longitudes = workout.get('longitude', [])
    latitudes = workout.get('latitude', [])
    altitudes = workout.get('altitude', [])
    heart_rates = workout.get('heart_rate', [])
    timestamps = workout.get('timestamp', [])
    speeds = workout.get('speed', [])
    
    create_map(longitudes, latitudes)
    plot_heart_rate(heart_rates, timestamps)
    analyze_route(workout, latitudes, longitudes, altitudes, timestamps, heart_rates, workout.get('gender'), workout.get('sport'))

else:
    print("\nO dicionário de treinos está vazio.")